# Proyecto: 'CDMX Un aire no tan bueno'
## Licenciatura en Ciencia de Datos
### Instituto de Investigaciones en Matemáticas Aplicadas y en Sistemas (IIMAS)
### Universidad Nacional Autónoma de México

Integrantes:

- Ortega Ibarra Jaime Jesús
- Martiñón Luna Jonathan José

----
# Exploración de datos.

## Índice de contenidos
* [Importando librerías](#E1)
* [Lectura de Datos](#E2)
* [Generando Funciones](#E3)
* [Pre-procesando Datos](#E4)
* [Discriminando datos](#E5)
* [Describiendo los datos](#E6)
	 - [IMECA](#S1)
	 - [Calidad del aire](#S2)
	 - [Tiraderos Clandestinos](#S3)
	 - [Nivel de plomo](#S4)
	 - [Afluencia en el transporte](#S5)
	 - [Escuelas públicas](#S6)
	 - [Escuelas privadas](#S7)
	 - [Mercados](#S8)
	 - [Parquímetros](#S9)
	 - [Áreas Verdes](#S10)
	 - [Áreas Verdes GeoJson](#S11)
	 - [Ciclistas](#S12)
	 - [Estaciones de Ecobici](#S13)
	 - [Contaminantes](#S14)
* [Graficando Datos](#E7)

----
<a name="E1"></a>
## Importando librerías necesarias

In [121]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import geopandas as gpd
from tabulate import tabulate
import re

----
<a name="E2"></a>
## Leyendo los datos

In [229]:
# Imeca2020
imeca_2020_df = pd.read_csv('../Datos/imeca_2020.csv')
#Calidad del aire
calidad_aire_df = pd.read_csv('../Datos/prueba_datos_calidad_aire.csv')
# Tiraderos clandestinos
tiraderos_clandestinos_df = pd.read_csv('../Datos/tiraderos-clandestinos-al-cierre-de-2017.csv')
# Nivel de plomo
nivel_plomo_df = pd.read_csv('../Datos/red_manual_plomo.csv',skiprows=8)
# Afluencia transporte
afluencia_transport_df = pd.read_csv('../Datos/afluencia-preliminar-en-transporte-publico.csv')
# Escuelas públicas
escuelas_publicas_df = pd.read_csv('../Datos/Escuelas/escuelas-publicas.csv')
# Escuelas privadas
escuelas_privadas_df = pd.read_csv('../Datos/Escuelas/escuelas-privadas.csv')
# Mercados
mercados_df = pd.read_csv('../Datos/mercados-publicos.csv')
# Parquímetros
parquimetros_df = pd.read_csv('../Datos/prueba-parquimetros.csv')
# Áreas Verdes
areas_verdes_df = pd.read_csv('../Datos/cdmx_areas_verdes_2017.csv')
# Áreas verdes con coordenadas
areas_verdes_Geo_df = gpd.read_file("../Datos/cdmx_areas_verdes_2017.geojson")
# Contador ciclistas
ciclistas_df = pd.read_csv('../Datos/contador-ciclistas.csv')
# Estaciones_Ecobici
estaciones_Ecobici_df = pd.read_csv('../Datos/estaciones-de-ecobici.csv')

#------------------
#Nota: Es importante descomprimir el archivo para poder leerlo.
# Contaminantes
contaminantes_df = pd.read_csv('../Datos/Contaminantes/contaminantes.csv')

----
<a name="E3"></a>
## Generando Funciones

In [175]:
def obten_dic_value_counts(objeto):
    '''
    @Author: Jonathan Martiñón
    
    @Descripción: 
                Función encargada de generar un diccionario
                a partir de los valores contados por pd.value_Counts()
                devuelve un diccionario donde el indice es el 'key' y
                el valor es 'value'
    '''
    Alcaldias = ['Benito Juarez','Tlahuac','Xochimilco','Tlalpan','Venustiano Carranza',
            'Milpa Alta', 'Miguel Hidalgo','Iztapalapa','Azcapotzalco','Cuajimalpa',
            'Cuauhtemoc', 'Alvaro Obregon', 'Magdalena Contreras', 'Coyoacan',
            'Gustavo A. Madero','Iztacalco']
    
    indices = objeto.index
    valores = objeto.values
    
    diccionario = dict(zip(indices,valores))
    
    for alcaldia in Alcaldias:
        if alcaldia not in indices:
            diccionario.update({alcaldia:0})
    
    return diccionario

In [302]:
def obten_alcaldia(direccion):
    '''
    @Author: Jonathan Martiñón
    
    @Descripción
            Para el dataset de escuelas tenemos cierta estructura
            donde la delegación viene incluída en el Domicilio.
            "DELEGACION x,". La presente función se encarga
            de extraer esa delegación.
    '''
    # Si tiene DELEGACION, hacemos un split
    dire = re.split("DELEGACION ",direccion)
    
    # Si el split se logra, tendrá al menos 2 elementos
    if len(dire)>1:
        # Nos interesa la segunda parte, que lleva una coma seguida
        # Lo quitamos y ajustamos a formato title
        alcaldia=dire[1].split(",")[0].title()
        
        #Datos atípicos
        if alcaldia == 'Cuajimalpa De Morelos':
            alcaldia = 'Cuajimalpa'
            
        elif alcaldia == 'Azcapotzalco Obregon':
            alcaldia = 'Azcapotzalco'
        elif alcaldia == 'Alvaro Obregón':
            alcaldia = 'Alvaro Obregon'
            
    # En caso de no encontrar delegación
    else:
        alcaldia = "No aplica"
    return alcaldia

In [289]:
def elimina_acentos_alcaldia(alcaldia):
    '''
    @Autor: Jonathan Martiñón
    
    @Descripción: 
            Función encargada de eliminar acentos en vocales para 
            una cadena.
    @Ejemplo
    alcaldia = 'Benito Juárez'
    elimina_acentos_alcaldia(alcaldia)
    >>> 'Benito Juarez'
    '''
    
    tildes = {"á":"a","é":"e","í":"i","ó":"o"}
    alcaldia = alcaldia.lower()
    for i in range(len(alcaldia)):
        
        if alcaldia[i] in list(tildes.keys()):
            
            alcaldia = alcaldia.replace(alcaldia[i],tildes[alcaldia[i]])
            
    return alcaldia.title()

----
<a name="E4"></a>
## Pre-procesamiento

In [9]:
data_df = pd.DataFrame()
data_df.head()

""


In [306]:
print(list(contaminantes_df.columns))
print(contaminantes_df.shape)
contaminantes_df.head()

['Fecha', 'Nombre de estación', 'Clave de estación', 'Clave Parámetro', 'Nombre de parámetro', 'Unidades parámetros', 'Valor', 'ID Unidad', 'Clave de unidad', 'Nombre de unidad', 'Altura Estación', 'Observación Estación', 'ID Estación', 'Latitud', 'Longitud', 'Coordenadas']
(469560, 16)


,Fecha,Nombre de estación,Clave de estación,Clave Parámetro,Nombre de parámetro,Unidades parámetros,Valor,ID Unidad,Clave de unidad,Nombre de unidad,Altura Estación,Observación Estación,ID Estación,Latitud,Longitud,Coordenadas
0,2020-02-15T20:00:00-06:00,Santa fe,SFE,NO2,Dióxido de nitrógeno,1,18.0,1,ppb,partes por billón,2599,NaN,484090040309,19.357357,-99.262865,"19.357357,-99.262865"
1,2020-02-15T20:00:00-06:00,Nezahualcóyotl,NEZ,NO2,Dióxido de nitrógeno,1,27.0,1,ppb,partes por billón,2235,NaN,484150580115,19.393734,-99.028212,"19.393734,-99.028212"
2,2020-02-15T20:00:00-06:00,San Agustín,SAG,O3,Ozono,1,8.0,1,ppb,partes por billón,2241,NaN,484150330327,19.532968,-99.030324,"19.532968,-99.030324"
3,2020-02-15T20:00:00-06:00,Villa de las Flores,VIF,O3,Ozono,1,6.0,1,ppb,partes por billón,2242,NaN,484150200109,19.658223,-99.096590,"19.658223,-99.09659"
4,2020-02-15T20:00:00-06:00,Milpa Alta,MPA,O3,Ozono,1,24.0,1,ppb,partes por billón,2594,NaN,484090090104,19.176900,-98.990189,"19.1769,-98.990189"


In [308]:
contaminantes_df['Nombre de estación'].unique()

array(['Santa fe', 'Nezahualcóyotl', 'San Agustín', 'Villa de las Flores',
       'Milpa Alta', 'Mguel Hidalgo', 'Pedregal', 'Acolman', 'Montecillo',
       'Ajusco Medio', 'Coyoacán', 'Centro de Ciencias de la Atmósfera',
       'Tultitlán', 'FES Aragón', 'Tlahuac', 'Atizapan', 'Chalco',
       'UAM Xochimilco', 'Investigaciones Nucleares', 'La Presa',
       'UAM Iztapalapa', 'Benito Juárez', 'Gustavo A. Madero',
       'San Juan Aragón', 'Xalostoc', 'Santiago Acahualtepec',
       'Camarones', 'Hospital General de México', 'Iztacalco',
       'Tlalnepantla', 'Cuajimalpa', 'FES Acatlán', 'Cuautitlán',
       'Los Laureles', 'Merced', 'Ajusco'], dtype=object)

In [301]:
# Obtenemos la cantidad de áreas verdes por delegación
areas_verdes_dic = obten_dic_value_counts(areas_verdes_df['delegacion'].apply(elimina_acentos_alcaldia).replace('Cuajimalpa De Morelos','Cuajimalpa').value_counts())

In [298]:
#Obtenemos la cantidad de parquímetros por alcaldía
parquimetros_dic = obten_dic_value_counts(parquimetros_df['Alcaldía'].apply(elimina_acentos_alcaldia).value_counts())

In [261]:
#Obtenemos la cantidad de mercados por alcaldía
mercados_dic = obten_dic_value_counts(mercados_df['alcaldia'].apply(str.title).value_counts())

In [253]:
# Obtenemos la cantidad de escuelas privadas por alcaldia
escuelas_privadas_dic = obten_dic_value_counts(escuelas_privadas_df['ALCALDÍA'].apply(str.title).replace('Cuajimalpa De Morelos',
                                                          'Cuajimalpa').replace('La Magdalena Contreras',
                                                                                'Magdalena Contreras').value_counts())

In [232]:
#Obtenemos la Alcaldía a partir del domicilio
escuelas_publicas_df['Alcaldia'] = escuelas_publicas_df['Domicilio con nombre'].apply(obten_alcaldia)

#Eliminamos aquellos lugares que no corresponden a la CDMX
escuelas_publicas_df.drop(escuelas_publicas_df[escuelas_publicas_df['Alcaldia']=='No aplica'].index
                          ,axis=0, inplace=True)

In [224]:
escuelas_publicas_dic = obten_dic_value_counts(escuelas_publicas_df['Alcaldia'].value_counts())

In [176]:
tiraderos_dic = obten_dic_value_counts(tiraderos_clandestinos_df.groupby('alcaldia')['alcaldia'].count())

----
<a name="E5"></a>
## Discriminando Datos

De los valores que estamos seguros que nos servirán será la alcaldía